# Script d'insertion des données dans la table "rankings"

In [ ]:
import pandas as pd
import mysql.connector

from datetime import datetime

In [ ]:
from config import config

In [ ]:
rankings_path = "../data/games_ranking.csv"

df_rankings = pd.read_csv(rankings_path)

In [ ]:
df_rankings

### Préparation des données avant insertion

#### Ajout des game_id

In [ ]:
# Connexion à la base

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [ ]:
# Récupération de la map game > id

cursor.execute("SELECT id, name FROM games")
game_id_map = {game: id_ for id_, game in cursor.fetchall()}

In [ ]:
# Ajout du game_id dans le dataframe des rankings

df_rankings['game_id'] = df_rankings['game_name'].map(game_id_map).astype('Int64')

#### Ajout des user_id

In [ ]:
# Récupération de la map genre > id

cursor.execute("SELECT id, ranking_genre FROM ranking_genres")
genre_id_map = {genre: id_ for id_, genre in cursor.fetchall()}

In [ ]:
# Ajout du ranking_genre_id dans le dataframe des rankings

df_rankings['ranking_genre_id'] = df_rankings['genre'].map(genre_id_map).astype('Int64')

#### Ajout des type_id

In [ ]:
# Récupération de la map type > id

cursor.execute("SELECT id, ranking_type FROM ranking_types")
type_id_map = {type: id_ for id_, type in cursor.fetchall()}

In [ ]:
# Ajout du ranking_type_id dans le dataframe des rankings

df_rankings['ranking_type_id'] = df_rankings['rank_type'].map(type_id_map).astype('Int64')

In [ ]:
# Fermeture de la connexion

cursor.close()
cnx.close()

In [ ]:
df_rankings

### Dernières préparation des données avant insertion

In [ ]:
df_rankings = df_rankings[['game_id', 'ranking_genre_id', 'ranking_type_id', 'rank']]

In [ ]:
# On supprime les objets dont un (ou plusieurs) des champs listés n'est pas renseigné

df_rankings = df_rankings.dropna(subset=['game_id', 'ranking_genre_id', 'ranking_type_id', 'rank'])

In [ ]:
# On renomme les colonnes pour correspondre à celles dans la table MySQL

df_rankings = df_rankings.rename(columns={'rank': 'ranking'})

### Insertion des données

In [ ]:
# Requête SQL

insert_query = """
    INSERT IGNORE INTO rankings
    (game_id, ranking_genre_id, ranking_type_id, ranking)
    VALUES (%s, %s, %s, %s)
"""

In [ ]:
# Connexion à la base

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [ ]:
# Exécution de la requête

for _, row in df_rankings.iterrows():
    cursor.execute(insert_query, tuple(row))

cnx.commit()

In [ ]:
# Fermeture de la connexion

cursor.close()
cnx.close()